### Библиотеки

In [ ]:
import pickle
from Experiment_Burgers_1 import *

In [ ]:
import sys
sys.path.append('D:\myProjects\diplom_2\GIT_stencil-net\GIT_stencil-net')
from utils import burgers_simulation,burgers_upwind

In [ ]:
BC_TYPE='dirichlet'; BC_VALUES=[0,0]

### Описание эксперимента:

- Полностью с параметрами автора

### 0. Параметры для генерации данных

In [ ]:
folder='Experiment_burgers_1_optuna'

In [ ]:
# для сил
np.random.seed(0)

L = 2.0*np.pi; T_ = 40;  Al = -0.1; Ar = 0.1; wl = -0.4; wr = 0.4; N = 20
A = np.zeros((N,)); w = np.zeros((N,)); phi = np.zeros((N,)); l = np.zeros((N,))

for k in range(0, N):
    A[k] = (Ar-Al)*np.random.rand(1) + Al
    w[k] = (wr-wl)*np.random.rand(1) + wl
    phi[k] = (2.0*np.pi)*np.random.rand(1)
    l[k] = np.random.randint(2, 5)

In [ ]:
# epochs_lst=[1,10,100,1_000,5_000,10_000]
epochs_lst=[100]

In [ ]:
display(main_params)

### 1. Генерация данных

In [ ]:
# n = 256; Tsim = 80001; tau = 0.01
# n = 256; Tsim = 16001; tau=0.01


x_lst = np.linspace(0,L,n); h = x_lst[1] - x_lst[0]; #D = 0.02; 

CFL = (h*h)/(2.0*D)
print("CFL dt", CFL, "actual dt", tau)

In [ ]:
# ##generate (by WENO + RK3)
# v_fact, phase = burgers_simulation(Tsim, n, x_lst, D, tau, A, w, phi, l, N, L,bc_type=BC_TYPE,bc_values=BC_VALUES)

# ##save
# pickle.dump(v_fact, open(fr'{folder}/data/v_fact_n={n}_Tsim={Tsim}.pkl', 'wb'))
# pickle.dump(phase, open(fr'{folder}/data/phase_n={n}_Tsim={Tsim}.pkl', 'wb'))

##load
# v_fact=pickle.load( open(fr'{folder}/data/v_fact_n={n}_Tsim={Tsim}.pkl', 'rb'))
# phase=pickle.load( open(fr'{folder}/data/phase_n={n}_Tsim={Tsim}.pkl', 'rb'))

In [ ]:
# ##generate (by UPWIND)
# v_fact = burgers_upwind(Tsim, n, x_lst, D, tau, A, w, phi, l, N, L,bc_type=BC_TYPE,bc_values=BC_VALUES)

#save
# pickle.dump(v_fact, open(fr'{folder}/data/v_fact_UPWIND_n={n}_Tsim={Tsim}.pkl', 'wb'))

##load
# v_fact=pickle.load( open(fr'{folder}/data/v_fact_UPWIND_n={n}_Tsim={Tsim}.pkl', 'rb'))

In [ ]:
v_fact = pickle.load( open(fr'{folder}/data/v_fact_CD_n={n}_Tsim={Tsim}.pkl', 'rb')).to_device('cpu')

In [ ]:
v_fact.shape

In [ ]:
#view
plt.plot(v_fact[:,0],'o-')
plt.plot(v_fact[:,-1],'.-')
plt.grid()
plt.title('Фактические данные')

In [ ]:
print('h',h,h**2)
print('tau',tau)
assert tau/h<=1,'не выполнено ограничение на число Куранта'

### 2. Сабсемплинг

In [ ]:
t_factor = 1  # time subsample factor 
s_factor = 1  # space subsample factor

# t_factor = 20
# s_factor = 4

# t_factor = 20
# s_factor = 5

Tsim=len(v_fact[0])
dxc,dtc,coarse_t,coarse_x,v_coarse,Lxc,\
Ltc,v_coarse_train,v_coarse_test=subsampling(s_factor,t_factor,h,tau,Tsim,n,v_fact,train_split=train_split)

In [ ]:
dxc==dtc, dxc

In [ ]:
##check right point
plt.figure()
plt.plot(v_coarse_train[0,:])
plt.plot(v_coarse_train[-1,:],'--')

plt.figure()
plt.plot(v_coarse_test[0,:])
plt.plot(v_coarse_test[-1,:],'--')

In [ ]:
plt.figure()
time=0 #first time
plt.plot(x_lst[coarse_x],v_coarse[:,time],'*',color='red',label=str(s_factor) + 'x FACT')
plt.plot(x_lst,v_fact[:,t_factor*time],'.',color='blue',label='FACT')
time=v_coarse.shape[1]-1 #last time
plt.plot(x_lst[coarse_x],v_coarse[:,time],'*',color='red',label=str(s_factor) + 'x FACT')
plt.plot(x_lst,v_fact[:,t_factor*time],'.',color='blue',label='FACT')
plt.legend()
# plt.title(" Time slice at t=0 and t = " + str(time))
# plt.show()
plt.title('Фактические данные после сабсемплинга')

### 3.1 Optuna

In [ ]:
import optuna
from sklearn.metrics import root_mean_squared_error

In [ ]:
N_TRIALS

In [ ]:
from utils import forcing_terms

Fc, Fc_0p5, Fc_p1, Fc_0m5, Fc_m1 = forcing_terms(A, w, phi, l, L, Lxc, T_, Ltc, N, dtc)

# u_train = torch.tensor(v_coarse_train.T, requires_grad=True, dtype=torch.float, device=device)
fc      = torch.tensor(Fc.T, requires_grad=True, dtype=torch.float, device=device)
fc_0p5  = torch.tensor(Fc_0p5.T, requires_grad=True, dtype=torch.float, device=device)
fc_p1   = torch.tensor(Fc_p1.T, requires_grad=True, dtype=torch.float, device=device)
fc_0m5  = torch.tensor(Fc_0m5.T, requires_grad=True, dtype=torch.float, device=device)
fc_m1   = torch.tensor(Fc_m1.T, requires_grad=True, dtype=torch.float, device=device)

In [ ]:
v_coarse_test.shape

In [ ]:
v_coarse_test_1=v_coarse_test[:,:1000]
v_coarse_test_1.shape

In [ ]:
def objective_mplconv(trial):
    # my_doc_params={
    # 'fs' : trial.suggest_categorical('fs',[1,2,3,4,5,6,7]),
    # 'neurons' : trial.suggest_categorical('neurons',[8,16,24,32,40,48,64]) ,
    # 'hidden_layers_num': trial.suggest_categorical('hidden_layers_num',[0,1,2,3,4]),
    # 'lr': trial.suggest_categorical('lr',[1e-02,1e-03]),
    # 'm' : trial.suggest_categorical('m',[2,4,6]) ,  
    # 'has_backward' : trial.suggest_categorical('has_backward',[True]),  
    # 'method' : trial.suggest_categorical('method',['RK3','E1']),
    # 'decay_const': trial.suggest_categorical('decay_const',[0.2,0.5,0.9])
    # }
    my_doc_params={
    'fs' : trial.suggest_categorical('fs',[3,5]),
    'neurons' : trial.suggest_categorical('neurons',[8,16,32]) ,
    'hidden_layers_num': trial.suggest_categorical('hidden_layers_num',[0,1,2,3]),
    'lr': trial.suggest_categorical('lr',[1e-01,1e-02]),
    'm' : trial.suggest_categorical('m',[1,2,4]) ,  
    'has_backward' : trial.suggest_categorical('has_backward',[True,False]),  
    'method' : trial.suggest_categorical('method',['E1','RK3']),
    'decay_const': trial.suggest_categorical('decay_const',[0.9])
    }
    
    ## loss на последней эпохе
    # metric=mf.train_net(mf.MLPConv,v_coarse_train,mf.epochs,dtc,
                        # **my_doc_params)[2]
    
    ## was
    # ## средние лосс с 10 последних итераций
    # metric_lst=train_net(MLPConv,v_coarse_train,epochs,dtc,
    #                     **my_doc_params,
    #                     force_terms=[fc, fc_0p5,fc_p1,fc_0m5,fc_m1],
    #                     bc_type=BC_TYPE,bc_values=BC_VALUES)[1]
    
    # metric_lst=np.array(metric_lst)[:,3][-10:]
    # metric=np.mean(metric_lst)
    

    ## new
    # train
    temp_net,_,_=train_net(MLPConv,v_coarse_train,epochs,dtc,
                        **my_doc_params,
                        force_terms=[fc, fc_0p5,fc_p1,fc_0m5,fc_m1],
                        bc_type=BC_TYPE,bc_values=BC_VALUES)
    # 
    NN_sim,T_sim,x_sim=make_simulation_forcing(temp_net,v_coarse_test_1,L,Lxc,dtc,\
                                              A,phi,N,w,l,method=method,
                                              bc_type=BC_TYPE,bc_values=BC_VALUES)


    print('Проверка на пропуски :',np.isnan(np.min(NN_sim)))

    if np.isnan(np.min(NN_sim))==False:

        metric = root_mean_squared_error(v_coarse_test_1,NN_sim)
        print(metric)
    else:
        metric = np.inf

    return metric

In [ ]:
v_coarse_test_1.shape,v_coarse_test.shape

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_mplconv, n_trials=N_TRIALS)

In [ ]:
study.trials_dataframe()
print('Best hyperparameters:', study.best_params)
print('Best RMSE:', study.best_value)

In [ ]:
my_doc_params=study.best_params

### 3.2 Тренировка модели

In [ ]:
from utils import forcing_terms

Fc, Fc_0p5, Fc_p1, Fc_0m5, Fc_m1 = forcing_terms(A, w, phi, l, L, Lxc, T_, Ltc, N, dtc)

# u_train = torch.tensor(v_coarse_train.T, requires_grad=True, dtype=torch.float, device=device)
fc      = torch.tensor(Fc.T, requires_grad=True, dtype=torch.float, device=device)
fc_0p5  = torch.tensor(Fc_0p5.T, requires_grad=True, dtype=torch.float, device=device)
fc_p1   = torch.tensor(Fc_p1.T, requires_grad=True, dtype=torch.float, device=device)
fc_0m5  = torch.tensor(Fc_0m5.T, requires_grad=True, dtype=torch.float, device=device)
fc_m1   = torch.tensor(Fc_m1.T, requires_grad=True, dtype=torch.float, device=device)

In [ ]:
for epochs in epochs_lst:
    
    net, loss_lst,loss=train_net(MLPConv,v_coarse_train,epochs,dtc,
              **my_doc_params,
              force_terms=[fc,fc_0p5,fc_p1,fc_0m5,fc_m1],
              bc_type=BC_TYPE,bc_values=BC_VALUES)
    
    pickle.dump(net, open(fr'{folder}/models/net_epochs={epochs}', 'wb'))
    

In [ ]:
high_epochs=1_000
my_doc_params=study.best_params
my_doc_params.update({'epoch':high_epochs})
display(make_param_table(net,my_doc_params,tau,h,n,t_factor,s_factor))

In [ ]:
# save study history
study.trials_dataframe().to_excel(fr'{folder}/models/trials_dataframe_n=' +str(n)+'_'+ str(list(my_doc_params.values()))+'.xlsx',index=False)

# save all hyperparams
make_param_table(net,my_doc_params,tau,h,n,t_factor,s_factor).to_excel(fr'{folder}/models/all_best_params_n='+str(n)+'_'+str(list(my_doc_params.values()))+'.xlsx',index=False)


# save hyperparams
my_doc_params['fs']=[my_doc_params['fs']]
pd.DataFrame.from_dict(my_doc_params,dtype=str)
pd.DataFrame.from_dict(my_doc_params,dtype=str).to_excel(fr'{folder}/models/best_params_n=' +str(n)+'_'+str(list(my_doc_params.values()))+'.xlsx',index=False)

In [ ]:
del my_doc_params['epoch']
my_doc_params['fs']=my_doc_params['fs'][0]
net,loss_lst,loss=train_net(MLPConv,v_coarse_train,high_epochs,dtc,**my_doc_params,
                            force_terms=[fc, fc_0p5,fc_p1,fc_0m5,fc_m1],
                            bc_type=BC_TYPE,bc_values=BC_VALUES)

pickle.dump(net, open(fr'{folder}/models/net_epochs={high_epochs}_device={device}', 'wb'))

In [ ]:
epochs_lst=[high_epochs]

### 4. Симуляция прогноза моделью

In [ ]:
NN_sim_lst=[]

weights=[]
biases=[]

for epochs in epochs_lst:
    
    net = pickle.load(open(fr'{folder}/models/net_epochs={epochs}_device={device}', 'rb'))
    
    # NN_sim,T_sim,x_sim=make_simulation(net,v_coarse,L,Lxc,dtc,method=method)
    NN_sim,T_sim,x_sim=make_simulation_forcing(net,v_coarse,L,Lxc,dtc,\
                                              A,phi,N,w,l,method=method,
                                              bc_type=BC_TYPE,bc_values=BC_VALUES)
    
    NN_sim_lst.append(NN_sim)
    
    weights.append(net.layer[0].weight.data.numpy())
    biases.append(None)

In [ ]:
method

In [ ]:
pd.DataFrame({'epoch':epochs_lst,#+[['finite_diff_methods']],
              'coefs':weights,#+[[1/((dxc)),-1/(dxc),0]],
              'bias':biases})#+[[None]]})

In [ ]:
print(dxc)

### 5. Результаты

In [ ]:
T=round(T_sim*dtc,3)
T

In [ ]:
err=np.abs(v_coarse-NN_sim)
mae_list=[err[:,i].mean() for i in range(err.shape[1])]
mae_df=pd.DataFrame({'method':method,'mae_list':mae_list}).to_csv(fr'{method}_mae.csv')

In [ ]:
method

In [ ]:
display(main_params)

In [ ]:

for i in range(len(epochs_lst)):
    
    print(fr'---------------Epochs = {epochs_lst[i]}-------------------')
    view_results(T_sim,x_sim,NN_sim_lst[i],v_coarse,T,dtc,n=11,fix_axes=False,
                epochs=epochs_lst[i],
                save_path = fr'{folder}/graphs/',
                save_name = fr'Result_epoch={epochs_lst[i]}',
                view_flag=False)
    view_result_metric(NN_sim_lst[i],v_coarse,T,dtc,L,dxc,
                       n_xticks=100,
                       save_path = fr'{folder}/graphs/',
                save_name = fr'Metric_epoch={epochs_lst[i]}')

    view_result_imshow(NN_sim_lst[i],v_coarse,T,dtc,L,dxc,
                       figsize=(10,5),aspect=10,n_xticks=30,n_yticks=20,
                       save_path = fr'{folder}/graphs/',
                       save_name = fr'Imshow_epoch={epochs_lst[i]}',
                       fix_colorbar_axes=False,
                       colorbar_min_max=[-0.1,1.1],cmap='seismic'
                       )
    
    make_gif(folder=fr'{folder}/graphs/',epochs=epochs_lst[i])

    # make_gif(folder=fr'{folder}/graphs/',epochs=epochs_lst[i],format='mp4')

    make_subplot_graphs(NN_sim_lst[0], v_coarse, x_sim, T_sim, T, dtc,8,nx=4,ny=2,
                        save_flg=True,
                        save_path = fr'{folder}/graphs/')

    print('\n')
print('done')

In [ ]:
# diff=NN_sim_lst[0]-v_coarse

# for j in range(len(diff[0,:])): 
#     if max(diff[:,j])!=0:
#         plt.figure()
#         plt.plot(diff[:,j],'-*')
#         plt.title(str(j)+' / '+str(len(diff[0,:])))
#         plt.grid()
        
#         plt.figure()
#         plt.plot(NN_sim_lst[0][:,j],'-*',label='NN_sim')
#         plt.plot(v_coarse[:,j],'-*',label='v_coarse')
#         plt.legend()
#         plt.grid()
        
#         break

In [ ]:
# for j in range(5,7):
#         # plt.plot(NN_sim_lst[0][:,j],'-*',color='red',label=fr'NN_sim_{j}')
#         # plt.plot(v_coarse[:,j],'--*',color='blue',label=fr'v_coarse_{j}')
#         plt.plot(v_fact[:,j],'--*',color='green',label=fr'v_coarse_{j}')
#         plt.legend()
#         plt.grid()

### RK3_mae vs E1_mae

In [ ]:
rk3_mae=pd.read_csv(fr'RK3_mae.csv')
e1_mae=pd.read_csv(fr'E1_mae.csv')
plt.plot(rk3_mae['mae_list'],label='RK3')
plt.plot(e1_mae['mae_list'],label='E1')
plt.grid()
plt.legend()